In [1]:
import os
import random
import shutil
import cv2
import numpy as np
from tqdm import tqdm

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Amplification factors
amplification_factors = {
    "AK": 2,
    "DF": 7,
    "VASC": 7,
    "SCC": 3,
    "MEL": 0.4,
    "NV": 0.14,
    "BCC": 0.54,
    "BKL": 0.7
}

# Paths
input_base_dir = "Classified_Images"
output_base_dir = "Amplified_Images_1"
os.makedirs(output_base_dir, exist_ok=True)

def random_crop(image, crop_size=(224, 224)):
    """Randomly crops an image while keeping the center visible."""
    h, w = image.shape[:2]
    crop_h, crop_w = crop_size
    if h <= crop_h or w <= crop_w:
        return cv2.resize(image, crop_size)
    
    x = random.randint(0, w - crop_w)
    y = random.randint(0, h - crop_h)
    return image[y:y + crop_h, x:x + crop_w]

def random_flip(image):
    """Randomly flips the image horizontally."""
    return cv2.flip(image, flipCode=1) if random.random() > 0.5 else image

def random_translate(image, max_shift=20):
    """Randomly translates the image within the max shift range."""
    h, w = image.shape[:2]
    tx = random.randint(-max_shift, max_shift)
    ty = random.randint(-max_shift, max_shift)
    translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    return cv2.warpAffine(image, translation_matrix, (w, h), borderMode=cv2.BORDER_REFLECT)

def augment_image(image_path, output_folder, num_augmentations):
    """Performs random crop, flip, and translation while keeping colors intact."""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    for i in range(num_augmentations):
        augmented_img = random_crop(img)
        augmented_img = random_flip(augmented_img)
        augmented_img = random_translate(augmented_img)
        
        save_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(image_path))[0]}_aug_{i}.jpg")
        cv2.imwrite(save_path, cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR))

def amplify_images():
    image_counts = {}
    for dx_class, factor in amplification_factors.items():
        input_folder = os.path.join(input_base_dir, dx_class)
        output_folder = os.path.join(output_base_dir, dx_class)
        os.makedirs(output_folder, exist_ok=True)
        image_files = os.listdir(input_folder)
        
        if factor < 1:
            selected_images = random.sample(image_files, int(len(image_files) * factor))
            for image in selected_images:
                shutil.copy(os.path.join(input_folder, image), os.path.join(output_folder, image))
        elif factor == 1:
            for image in image_files:
                shutil.copy(os.path.join(input_folder, image), os.path.join(output_folder, image))
        else:
            for image in tqdm(image_files, desc=f"Augmenting {dx_class}"):
                augment_image(os.path.join(input_folder, image), output_folder, factor)
        
        image_counts[dx_class] = len(os.listdir(output_folder))
    return image_counts

new_image_counts = amplify_images()

print("\nNew image counts by class:")
for dx, count in new_image_counts.items():
    print(f"{dx}: {count} images")


Augmenting SCC: 100%|██████████| 628/628 [00:25<00:00, 24.71it/s]



New image counts by class:
AK: 1734 images
DF: 1673 images
VASC: 1771 images
SCC: 1884 images
MEL: 1808 images
NV: 1802 images
BCC: 1794 images
BKL: 1836 images
